In [1]:
IMAGE_SIZE = 224
NUM_CHANELLS = 3
LEN_FEATURES = IMAGE_SIZE * IMAGE_SIZE * NUM_CHANELLS
BATCH_SIZE = 64
# ZISE_OF_IMAGES = 4500
ZISE_OF_IMAGES = 3000
CLASSES = ['happy', 'sad']
# DRIVE_PATH = '/content/drive'
PATH = r"D:\\Projects\Development\\courses_projects\\deep_learning_and_natural_language_processing\\dog-image-mood-classification"
PATH_TO_DATA = PATH + '\\' + "data\\images"
RES_PATH = r"results\\"

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# tools
import os
import glob
import random
import numpy as np
from datetime import datetime
import pytz
import sklearn as sk
from tqdm import tqdm

# modeling
import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds
tf.disable_v2_behavior()
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:

def split_train_test(test_size: float, images, labels, files):
  size_of_train = round(len(images) * test_size)
  train_images = images[0: size_of_train]
  train_labels = labels[0: size_of_train]
  train_files = files[0: size_of_train]

  test_images = images[size_of_train: -1]
  test_labels = labels[size_of_train: -1]
  test_files = files[size_of_train: -1]

  return [train_images, train_labels, train_files], [test_images, test_labels, test_files]

def load_data(classes):
  images = []
  labels = []
  files = []
  for i in range(len(classes)):
    folder_name = classes[i]
    path = os.path.join(PATH_TO_DATA, folder_name)
    count = 0
    for file_index, file_name in enumerate(os.listdir(path)):
      images.append(file_name)
      label = [i]
      labels.append(label)
      files.append(os.path.join(path, file_name))
      if count == ZISE_OF_IMAGES:
        break
      count += 1

  return images, labels, files

def generate_batch(file_names, labels, batch_size):
  num_of_batch = int(len(file_names)/batch_size)
  i = 0
  while(True):
    if i == num_of_batch:
      i = 0
    start = i * batch_size
    end = (i + 1) * batch_size
    images = []
    for file_name in file_names[start:end]:
      image = cv2.imread(file_name)
      image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
      images.append(image)
    yield np.array(images), np.array(labels[start:end])
    i += 1

def normalise_data(data):
  data = data.astype(np.float32)
  for i in range(len(data)):
    for j in range(len(data[i])):
      data[i][j]/=255.0
  return data

def plot_images(images, cls_true, cls_pred=None):
    
    if len(images) == 0:
        print("no images to show")
        return 
    else:
        random_indices = random.sample(range(len(images)), min(len(images), 9))
        
    if cls_pred is not None:
        images, cls_true, cls_pred  = zip(*[(images[i], cls_true[i], cls_pred[i]) for i in random_indices])
    else:
        images, cls_true  = zip(*[(images[i], cls_true[i]) for i in random_indices])
    
    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i].reshape(IMAGE_SIZE, IMAGE_SIZE, 3))

        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true[i])
        else:
            xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])

        ax.set_xlabel(xlabel)
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

## Prepare data

In [4]:
images, labels, files = load_data(CLASSES)
images, labels, files = shuffle(images, labels, files)  # shuffle the data

In [5]:
train, test = split_train_test(test_size=0.8, images=images, labels=labels, files=files)
train_images, train_labels, train_files = train[:]
test_images, test_labels, test_files = test[:]

In [6]:
print(f"Size of train: {len(train_images)}")
print(f"Size of test: {len(test_images)}")

Size of train: 4802
Size of test: 1199


## Utils

In [7]:
eps = 1e-12
x = tf.placeholder(tf.float32, [None,  LEN_FEATURES])      
y_ = tf.placeholder(tf.float32, [None, 1])
W = tf.Variable(tf.zeros([LEN_FEATURES, 1]))
b = tf.Variable(tf.zeros([1]))
pred = 1 / (1.0 + tf.exp(-(tf.matmul(x,W) + b)))
loss1 = -(y_ * tf.log(pred + eps) + (1 - y_) * tf.log(1- pred + eps))
loss = tf.reduce_mean(loss1)
# update = tf.train.GradientDescentOptimizer(0.000000001).minimize(loss)
update = tf.train.AdamOptimizer(0.0001).minimize(loss)

train_loss_array = []
test_loss_array = []
epoch_array = []
y_logits = tf.matmul(x, W) + b
y_h = tf.round(tf.sigmoid(y_logits))


it_batch_train= generate_batch(train_files, train_labels, BATCH_SIZE)
it_batch_test = generate_batch(test_files, test_labels, BATCH_SIZE)
num_batch_train = round(len(train_images)/BATCH_SIZE)
num_batch_test = round(len(test_images)/BATCH_SIZE)
timezone = pytz.timezone("Israel")
now_time = datetime.now(timezone).strftime("%d-%m-%Y_%H-%M-%S")
path_to_res = f'{RES_PATH}res_{now_time}.txt'
epoch_array = []

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in tqdm(range(100)):
    train_loss, test_loss = 0, 0

    # Train
    for b in range(num_batch_train):
      data_x, data_y = next(it_batch_train)
      data_x = data_x.reshape(len(data_x), LEN_FEATURES)
      data_x = normalise_data(data_x)
      current_train_loss = sess.run([update, loss], feed_dict = {x:data_x, y_:data_y})[1]
      train_loss += current_train_loss
    train_loss /= num_batch_train

    # Test
    all_y_test = np.array([])
    all_y_pred = np.array([])

    for b in range(num_batch_test):
      test_x, test_y = next(it_batch_test)
      test_x = normalise_data(test_x)
      test_x = test_x.reshape(len(test_x), LEN_FEATURES)
      pred, current_loss = sess.run([y_h, loss], feed_dict = {x: test_x, y_: test_y})
      test_loss += current_loss

      y_pred = np.array(pred)[:,0]
      test_y = np.array(test_y)[:,0]
      all_y_pred = np.concatenate((all_y_pred, y_pred))
      all_y_test = np.concatenate((all_y_test, test_y))
    test_loss /= num_batch_test

    with open(path_to_res, 'a') as f:
      print(f"\nEpoch: {epoch} Loss train: {train_loss:.4}   Loss Test: {test_loss:.4}", file=f)
      print(sk.metrics.classification_report(all_y_test.astype(int),all_y_pred.astype(int), target_names=CLASSES), file=f)

    if epoch % 10 == 0:
      print(f"\nEpoch: {epoch} Loss train: {train_loss:.4}   Loss Test: {test_loss:.4}")
    train_loss_array.append(train_loss)
    test_loss_array.append(test_loss)
    epoch_array.append(epoch)
print(sk.metrics.classification_report(all_y_test.astype(int),all_y_pred.astype(int), target_names=CLASSES))

  1%|          | 1/100 [31:38<52:12:45, 1898.64s/it]


Epoch: 0 Loss train: 0.841   Loss Test: 0.6741


 11%|█         | 11/100 [5:10:09<41:57:35, 1697.26s/it]


Epoch: 10 Loss train: 0.6689   Loss Test: 0.6972


 21%|██        | 21/100 [10:11:59<38:59:14, 1776.64s/it]

...


Epoch: 40 Loss train: 0.5126   Loss Test: 0.816


 51%|█████     | 51/100 [23:43:44<17:54:18, 1315.48s/it]


Epoch: 50 Loss train: 0.4823   Loss Test: 0.8276


 51%|█████     | 51/100 [23:48:05<22:52:05, 1680.11s/it]


KeyboardInterrupt: 

In [ ]:
plt.title("Train loss value throw train logistic regression")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.plot(epoch_array, train_loss_array)
plt.show()

In [ ]:
plt.title("Test loss value throw train logistic regression")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.plot(epoch_array, test_loss)
plt.show()

In [ ]:
y_output = tf.matmul(x, W) + b
y_sigma = tf.round(tf.sigmoid(y_output))
y_pred = sess.run(y_sigma, feed_dict = {x: test_images, y_: test_labels})
print(classification_report(test_labels, y_pred))

In [ ]:
plot_images(test_images, test_labels, y_pred)